In [1]:
#Importing Libraries
import cv2 # used for image processing
import dlib # used for object detection
import time # fps
import math

In [2]:
#Classifier File
carCascade = cv2.CascadeClassifier("vech.xml") # contains a predtrained model

#Video file capture
video = cv2.VideoCapture("carsVideo.mp4") # video file

# Constant Declaration
WIDTH =1280
HEIGHT = 720

#estimate speed function
def estimateSpeed(location1, location2): # take 2 locations of the vehicle
    d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
    ppm = 8.8 # pixels per meter
    d_meters = d_pixels / ppm 
    fps = 18 # frames per second
    speed = d_meters * fps * 3.6 # converting to km/h
    return speed

#tracking multiple objects
def trackMultipleObjects():
    rectangleColor = (0, 255, 255) # green color rectangle
    frameCounter = 0
    currentCarID = 0
    fps = 0

    carTracker = {} # stores trackers 
    carLocation1 = {} # previous car location
    carLocation2 = {} # current car location
    speed = [None] * 1000 # speed of cars

    out = cv2.VideoWriter('outTraffic.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (WIDTH, HEIGHT))
    # mjpg -  motion jpeg , 10 - frames per second
    while True:
        start_time = time.time()
        rc, image = video.read() # read each frame
        if type(image) == type(None):
            break

        image = cv2.resize(image, (WIDTH, HEIGHT))
        resultImage = image.copy()

        frameCounter = frameCounter + 1 # increment frame counter
        carIDtoDelete = []

        for carID in carTracker.keys():
            trackingQuality = carTracker[carID].update(image) 

            if trackingQuality < 7:
                carIDtoDelete.append(carID) # delete car if tracking quality is less than 7

        
        for carID in carIDtoDelete: # delete low quality trackers
            print("Removing carID " + str(carID) + ' from list of trackers. ')
            print("Removing carID " + str(carID) + ' previous location. ')
            print("Removing carID " + str(carID) + ' current location. ')
            carTracker.pop(carID, None)
            carLocation1.pop(carID, None)
            carLocation2.pop(carID, None)

        
        if not (frameCounter % 10): # detect after 10 frames
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # gray scale
            cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24)) 
            # 1.1 - scale factor, 13 - min neighbours, 18 - min size of object, 24,24 - min size of object
            for (_x, _y, _w, _h) in cars:
                x = int(_x)
                y = int(_y)
                w = int(_w)
                h = int(_h)

                x_bar = x + 0.5 * w # center of x
                y_bar = y + 0.5 * h # center of y

                matchCarID = None 

                for carID in carTracker.keys():
                    trackedPosition = carTracker[carID].get_position()

                    t_x = int(trackedPosition.left())
                    t_y = int(trackedPosition.top())
                    t_w = int(trackedPosition.width())
                    t_h = int(trackedPosition.height())

                    t_x_bar = t_x + 0.5 * t_w
                    t_y_bar = t_y + 0.5 * t_h

                    if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
                        matchCarID = carID

                if matchCarID is None: # new car
                    print(' Creating new tracker' + str(currentCarID))

                    tracker = dlib.correlation_tracker() 
                    tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))

                    carTracker[currentCarID] = tracker
                    carLocation1[currentCarID] = [x, y, w, h]

                    currentCarID = currentCarID + 1

        for carID in carTracker.keys():
            trackedPosition = carTracker[carID].get_position()

            t_x = int(trackedPosition.left())
            t_y = int(trackedPosition.top())
            t_w = int(trackedPosition.width())
            t_h = int(trackedPosition.height())

            cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4) # draw rectangle

            carLocation2[carID] = [t_x, t_y, t_w, t_h] # update current location

        end_time = time.time() # end time

        for i in carLocation1.keys():
            if frameCounter % 1 == 0:
                [x1, y1, w1, h1] = carLocation1[i]
                [x2, y2, w2, h2] = carLocation2[i]

                carLocation1[i] = [x2, y2, w2, h2]

                if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                    if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                        speed[i] = estimateSpeed([x1, y1, w1, h1], [x1, y2, w2, h2])

                    if speed[i] != None and y1 >= 180:
                        cv2.putText(resultImage, str(int(speed[i])) + "km/h", (int(x1 + w1/2), int(y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 100) ,2)

        cv2.imshow('result', resultImage)

        out.write(resultImage)

        if cv2.waitKey(1) == 27:
            break

    
    cv2.destroyAllWindows()
    out.release()

if __name__ == '__main__':
    trackMultipleObjects()

 Creating new tracker0
 Creating new tracker1
 Creating new tracker2
 Creating new tracker3
 Creating new tracker4
 Creating new tracker5
 Creating new tracker6
Removing carID 0 from list of trackers. 
Removing carID 0 previous location. 
Removing carID 0 current location. 
 Creating new tracker7
 Creating new tracker8
 Creating new tracker9
Removing carID 7 from list of trackers. 
Removing carID 7 previous location. 
Removing carID 7 current location. 
Removing carID 9 from list of trackers. 
Removing carID 9 previous location. 
Removing carID 9 current location. 
Removing carID 1 from list of trackers. 
Removing carID 1 previous location. 
Removing carID 1 current location. 
 Creating new tracker10
 Creating new tracker11
 Creating new tracker12
Removing carID 4 from list of trackers. 
Removing carID 4 previous location. 
Removing carID 4 current location. 
Removing carID 10 from list of trackers. 
Removing carID 10 previous location. 
Removing carID 10 current location. 
Removing ca